In [52]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import CamembertForSequenceClassification, CamembertTokenizer

In [53]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [61]:
class TweetDataset(Dataset):
    def __init__(
        self, device, file_path="data/train.csv"
    ):
        f = open(file_path, "r")
        self.device = device
        self.data_lines = f.readlines()
        self.tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

    def __len__(self):
        return len(self.data_lines)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        line = self.data_lines[idx]
        arr = line.split(",")
        label = int(arr[0])
        
        sentence = ",".join(arr[1:])[:-1]
        inputs = self.tokenizer.encode_plus(sentence, max_length=100, pad_to_max_length=True)
        input_ids = torch.tensor([inputs["input_ids"]]).to(self.device)
        token_type_ids = torch.tensor([inputs["token_type_ids"]]).to(self.device)

        sample = {"input_ids": input_ids[0], "token_type_ids": token_type_ids[0], "label": label}

        return sample

In [62]:
train_dataset = TweetDataset(
    device,
    file_path="data/train.csv",
)

In [63]:
train_batch_size=20
dev_batch_size=10

train_dataloader = DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=0
)

In [64]:
model = CamembertForSequenceClassification.from_pretrained("camembert-base")
model.to(device)

CamembertForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [65]:
loss = torch.nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
for step, sample_batched in enumerate(train_dataloader):
    print(step)
    
    with torch.set_grad_enabled(True):
        print(sample_batched["input_ids"])
        output = model(sample_batched["input_ids"], token_type_ids = sample_batched["token_type_ids"])
        loss = loss(output, sample_batched["label"])
        self.optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    

0
tensor([[    5,  1156,     7,  ...,     1,     1,     1],
        [    5,   158,   349,  ...,     1,     1,     1],
        [    5,    74,  8738,  ...,     1,     1,     1],
        ...,
        [    5,   757,     7,  ...,     1,     1,     1],
        [    5,   821,   104,  ...,     1,     1,     1],
        [    5, 25847,    23,  ...,     1,     1,     1]])


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/simon/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-66-09a792c2dc36>", line 7, in <module>
    loss = loss(output, sample_batched["label"])
  File "/home/simon/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 532, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/simon/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py", line 601, in forward
    reduction=self.reduction)
  File "/home/simon/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py", line 2123, in binary_cross_entropy_wi